In [127]:
import pandas as pd
FICHEROS_CSV = [
            'data_dictionary', 'order_details',
            'orders', 'pizza_types', 'pizzas']

In [128]:
i = 3

df = pd.read_csv(FICHEROS_CSV[i] + '.csv', sep=',', encoding = 'cp1252')
string = df.dtypes.to_string()
mensaje = '\nEl fichero ' + FICHEROS_CSV[i]
mensaje += '.csv contiene las siguientes columnas:\n'
mensaje += '\n' + string +'\n'
file = open('analisis_datos.txt', 'w')
file.write(mensaje)
file.close()

In [129]:
string = df.dtypes.to_dict()
for key in string.keys():
    string[key] = str(string[key])
print(string)
dicc = {FICHEROS_CSV[i] + '.csv':string}
print(dicc)
df2 = pd.DataFrame(dicc)
df2

{'pizza_type_id': 'object', 'name': 'object', 'category': 'object', 'ingredients': 'object'}
{'pizza_types.csv': {'pizza_type_id': 'object', 'name': 'object', 'category': 'object', 'ingredients': 'object'}}


,pizza_types.csv
category,object
ingredients,object
name,object
pizza_type_id,object


In [130]:
file = open('analisis_datos.txt', 'w')
file.close()

for archivo in FICHEROS_CSV:

    df = pd.read_csv(archivo + '.csv', sep=',', encoding = 'cp1252')

    mensaje = '\nEl fichero ' + archivo
    mensaje += '.csv contiene las siguientes columnas:\n\n'

    for colum in df.columns:
        tmp = colum
        tmp += '    ' + str(df[colum].dtype)
        tmp += '    Valores Null/Nan = ' + str(df[colum].isnull().sum())
        mensaje += tmp + '\n'

    file = open('analisis_datos.txt', 'a')
    file.write(mensaje)
    file.close()
